In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# BigQuery DataFrames (BigFrames) AI Functions

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/generative_ai/ai_functions.ipynb">
      <img src="https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/refs/heads/main/third_party/logo/colab-logo.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/generative_ai/ai_functions.ipynb">
      <img src="https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/refs/heads/main/third_party/logo/github-logo.png" width="32" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/generative_ai/ai_functions.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35">
      Open in BQ Studio
    </a>
  </td>
</table>

This notebooks provides examples on how to use BigFrames AI functions, which are offered under the `bigframes.bigquery` package. 

## Preparation

First, set up your BigFrames environment

In [3]:
import bigframes.pandas as bpd 

PROJECT_ID = "bigframes-dev" # Your project ID here

bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.ordering_mode = "partial"
bpd.options.display.progress_bar = None

## ai.generate

The `ai.generate` function lets you analyze any combination of text and unstructured data from BigQuery. You can mix BigFrames/Pandas series with string literals as your prompt in the form of a tuple. Here is an example:

In [5]:
import bigframes.bigquery as bbq

ingredients1 = bpd.Series(["Lettuce", "Sausage"])
ingredients2 = bpd.Series(["Cucumber", "Long Bread"])

prompt = ("What's the food made from ", ingredients1, " and ", ingredients2, " One word only")
bbq.ai.generate(prompt)

0    {'result': 'Salad\n', 'full_response': '{"cand...
1    {'result': 'Hotdog\n', 'full_response': '{"can...
dtype: struct<result: string, full_response: extension<dbjson<JSONArrowType>>, status: string>[pyarrow]

The function returns a series of structs. The `'result'` field holds the answer, while more metadata can be found in the `'full_response'` field. The `'status'` field tells you whether LLM made a successful response for that specific row. 

You can also include additional model parameters into your function call, as long as they satisfy the structure of `generateContent` [request body format](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.endpoints/generateContent#request-body)  For example:

In [ ]:
model_params = {
    "generationConfig": {"maxOutputTokens": 1}
}

ingredients1 = bpd.Series(["Lettuce", "Sausage"])
ingredients2 = bpd.Series(["Cucumber", "Long Bread"])

prompt = ("What's the food made from ", ingredients1, " and ", ingredients2)
bbq.ai.generate(prompt).struct.field("result")

1    The food made from sausage and long bread is m...
0    Lettuce and cucumber are commonly used togethe...
Name: result, dtype: string